In [2]:
# Do all imports and installs here
! pip install boto3
import numpy as np
import pandas as pd
import os
from sys import path
import time
import boto3
import json
import configparser
import logging
from pyspark.sql import SparkSession
from pyspark.sql.types import DateType
from pyspark.sql.functions import udf, col

config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

KEY                    = config.get('AWS','KEY')
SECRET                 = config.get('AWS','SECRET')

# setup logging 
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Read AWS configurations
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = "/opt/conda/bin:/opt/spark-2.4.3-bin-hadoop2.7/bin:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/lib/jvm/java-8-openjdk-amd64/bin"
os.environ["SPARK_HOME"] = "/opt/spark-2.4.3-bin-hadoop2.7"
os.environ["HADOOP_HOME"] = "/opt/spark-2.4.3-bin-hadoop2.7"

os.environ['AWS_ACCESS_KEY_ID']= KEY
os.environ['AWS_SECRET_ACCESS_KEY']= SECRET

spark = SparkSession.builder\
        .config("spark.jars.repositories", "https://repos.spark-packages.org/")\
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0,saurfang:spark-sas7bdat:2.0.0-s_2.11")\
        .enableHiveSupport().getOrCreate()

bucket_name = 'joyimmigrant' # already created on S3

In [ ]:
#move sas file to S3 already
#move sas file from space to s3
# s3_resource = boto3.client('s3',
#                        region_name="us-west-2",
#                        aws_access_key_id=KEY,
#                        aws_secret_access_key=SECRET
#                    )
# sasfile ='../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'

# s3_resource.upload_file(sasfile,bucket_name,
#     'rawdata/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')

In [3]:
#read and handle immigrant data
sasfile = 'rawdata/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'

sasfile = 's3://'+bucket_name+'/'+sasfile
df_spark = spark.read.format('com.github.saurfang.sas.spark') \
    .load(sasfile)
df_spark = df_spark.select('cicid')
df_spark.show(2)



#save to S3
df_spark.write.mode("overwrite") \
        .parquet('s3://'+bucket_name + "/data/immigrant_table.parquet")


Py4JJavaError: An error occurred while calling o45.load.
: java.io.IOException: No FileSystem for scheme: s3
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:2660)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:2667)
	at org.apache.hadoop.fs.FileSystem.access$200(FileSystem.java:94)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:2703)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:2685)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:373)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:295)
	at com.github.saurfang.sas.spark.SasRelation.inferSchema(SasRelation.scala:114)
	at com.github.saurfang.sas.spark.SasRelation.<init>(SasRelation.scala:50)
	at com.github.saurfang.sas.spark.SasRelation$.apply(SasRelation.scala:42)
	at com.github.saurfang.sas.spark.DefaultSource.createRelation(DefaultSource.scala:50)
	at com.github.saurfang.sas.spark.DefaultSource.createRelation(DefaultSource.scala:39)
	at